# Time Series with Polars on the GPU

In [ ]:
!nvidia-smi | head

Mon Dec 16 22:34:19 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   41C    P8              12W /  72W |      1MiB / 23034MiB |      0%      Default |


Install with UV

In [ ]:
!uv pip install --system  -U altair hvplot

Using Python 3.10.12 environment at /usr
Resolved 43 packages in 1.05s
⠙ Preparing packages... (0/9)
⠙ Preparing packages... (0/9)
⠙ Preparing packages... (0/9)
attrs      ------------------------------     0 B/61.91 KiB
⠙ Preparing packages... (0/9)
attrs      ------------------------------ 14.88 KiB/61.91 KiB
⠙ Preparing packages... (0/9)
attrs      ------------------------------ 14.88 KiB/61.91 KiB
python-dateutil ------------------------------ 14.86 KiB/224.50 KiB
⠙ Preparing packages... (0/9)
attrs      ------------------------------ 14.88 KiB/61.91 KiB
python-dateutil ------------------------------ 14.86 KiB/224.50 KiB
⠙ Preparing packages... (0/9)
attrs      ------------------------------ 14.88 KiB/61.91 KiB
python-dateutil ------------------------------ 30.86 KiB/224.50 KiB
⠙ Preparing packages... (0/9)
attrs      ------------------------------ 14.88 KiB/61.91 KiB
certifi    ------------------------------     0 B/161.06 KiB
python-dateutil ------------------------------ 30.86 K

In [ ]:
!uv pip install --system -U --extra-index-url=https://pypi.nvidia.com  polars[gpu]

Using Python 3.10.12 environment at /usr
Resolved 15 packages in 1.50s
Audited 15 packages in 0.20ms


## Getting Data

In [ ]:

!if [ ! -f "usa_stocks_30m.parquet" ]; then curl https://storage.googleapis.com/rapidsai/colab-data/usa_stocks_30m.parquet -o usa_stocks_30m.parquet; else echo "usa_stocks_30m.parquet found"; fi


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  633M  100  633M    0     0  15.8M      0  0:00:40  0:00:40 --:--:-- 19.6M


In [ ]:
import polars as pl
import polars.selectors as cs

pl.__version__

'1.14.0'

In [ ]:
stocks_lazy = pl.scan_parquet("usa_stocks_30m.parquet")

In [ ]:
stocks_lazy

## Check Types

In [ ]:
(stocks_lazy
 .head(10)
 .collect(engine='gpu'))

__index_level_0__,datetime,open,high,low,close,volume,ticker
i64,datetime[ms],f64,f64,f64,f64,i64,str
0,1999-11-18 17:00:00,45.56,50.0,45.5,46.0,9275000,"""A"""
1,1999-11-18 17:30:00,46.0,47.69,45.82,46.57,3200900,"""A"""
2,1999-11-18 18:00:00,46.56,46.63,41.0,41.0,3830500,"""A"""
3,1999-11-18 18:30:00,41.0,43.38,40.37,42.38,3688600,"""A"""
4,1999-11-18 19:00:00,42.31,42.44,41.56,41.69,1584300,"""A"""
5,1999-11-18 19:30:00,41.69,42.19,41.62,41.88,808800,"""A"""
6,1999-11-18 20:00:00,41.94,42.75,41.87,42.19,1211500,"""A"""
7,1999-11-18 20:30:00,42.19,42.94,42.06,42.82,728600,"""A"""
8,1999-11-18 21:00:00,42.88,42.88,42.25,42.31,775900,"""A"""


In [ ]:
(stocks_lazy
 .head(10)
 .collect(engine='gpu')
 .dtypes
 )

[Int64,
 Datetime(time_unit='ms', time_zone=None),
 Float64,
 Float64,
 Float64,
 Float64,
 Int64,
 String]

## Time Aggregations

In [ ]:
%%time
(stocks_lazy
 .with_columns(year=pl.col('datetime').dt.year())
 .group_by('year')
 .agg(cs.numeric().mean())
 .collect(engine='gpu')
 )

CPU times: user 357 ms, sys: 158 ms, total: 516 ms
Wall time: 326 ms


year,__index_level_0__,open,high,low,close,volume
i32,f64,f64,f64,f64,f64,f64
2002,12971.554929,36.646697,36.783138,36.508293,36.646054,263078.324184
2018,53810.03182,113.188075,113.505029,112.859166,113.18192,273294.390298
2000,7353.462272,47.018504,47.262723,46.773424,47.019192,220800.759298
2024,69888.33292,208.398297,208.859837,207.91459,208.411046,256332.253163
2012,38487.307259,55.84671,55.982971,55.71153,55.849,391315.550438
…,…,…,…,…,…,…
2021,61896.670541,186.129599,186.63599,185.607674,186.126191,258173.930134
2007,25657.922046,54.014047,54.17905,53.846799,54.013655,409269.848016
2008,27858.507801,44.856341,45.101187,44.606632,44.852959,591277.367665


In [ ]:
%%time
(stocks_lazy
 .with_columns(year=pl.col('datetime').dt.year())
 .group_by('year')
 .agg(cs.numeric().mean())
 .collect() # run on CPU
 )


CPU times: user 5.72 s, sys: 929 ms, total: 6.65 s
Wall time: 705 ms


year,__index_level_0__,open,high,low,close,volume
i32,f64,f64,f64,f64,f64,f64
2004,18736.99736,42.591014,42.691719,42.491388,42.593011,284706.816203
2007,25657.922046,54.014047,54.17905,53.846799,54.013655,409269.848016
2010,33572.067661,45.341668,45.470593,45.211743,45.343101,593204.191878
2015,45960.157136,84.434508,84.645613,84.221859,84.434671,284057.468152
2013,40964.630725,67.424388,67.57474,67.27526,67.42721,330078.350542
…,…,…,…,…,…,…
2018,53810.03182,113.188075,113.505029,112.859166,113.18192,273294.390298
1999,4568.234217,49.767378,49.959096,49.574967,49.767398,189990.686267
2016,48711.413372,84.730371,84.942757,84.522522,84.73313,295077.834475


In [ ]:
%%time
# groupby two items
(stocks_lazy
 .with_columns(year=pl.col('datetime').dt.year())
 .group_by(['ticker','year'])
 .agg(cs.numeric().mean())
 .collect(engine='gpu')
 )

CPU times: user 285 ms, sys: 112 ms, total: 396 ms
Wall time: 378 ms


ticker,year,__index_level_0__,open,high,low,close,volume
str,i32,f64,f64,f64,f64,f64,f64
"""EIX""",2003,17876.5,16.478191,16.526376,16.430221,16.480795,94742.125921
"""ITW""",2009,37466.0,38.656422,38.79906,38.51955,38.660826,280829.59198
"""PM""",2019,36748.5,81.164758,81.35803,80.974375,81.166366,289072.751838
"""CPT""",2018,66601.0,88.849166,89.035491,88.656787,88.846904,29964.65097
"""BK""",2000,8161.5,46.533491,46.737838,46.32557,46.531653,149850.153092
…,…,…,…,…,…,…,…
"""MCD""",2005,24421.0,31.900816,31.978499,31.825008,31.902018,440495.628248
"""CAG""",2011,44000.5,24.414817,24.465691,24.364893,24.416073,291631.645477
"""DVN""",2011,21960.5,75.831152,76.082619,75.571822,75.82721,246032.207824


In [ ]:
%%time
# sort results
(stocks_lazy
 .with_columns(year=pl.col('datetime').dt.year())
 .group_by(['ticker','year'])
 .agg(cs.numeric().mean())
 .sort(['ticker', 'year'])
 .collect(engine='gpu')
 )

CPU times: user 405 ms, sys: 148 ms, total: 553 ms
Wall time: 335 ms


ticker,year,__index_level_0__,open,high,low,close,volume
str,i32,f64,f64,f64,f64,f64,f64
"""A""",1999,189.0,47.51504,47.881187,47.208522,47.554354,275892.612137
"""A""",2000,2011.5,69.742293,70.320141,69.139541,69.718236,222762.461727
"""A""",2001,5247.0,33.358924,33.59634,33.139426,33.366602,170170.826833
"""A""",2002,8477.5,22.984785,23.118953,22.851465,22.981314,189420.178133
"""A""",2003,11733.5,20.174097,20.250863,20.0986,20.177482,156302.885135
…,…,…,…,…,…,…,…
"""ZTS""",2020,24192.5,144.723845,145.203939,144.254515,144.727793,92883.735671
"""ZTS""",2021,27528.5,189.112176,189.522102,188.718892,189.122105,76024.96138
"""ZTS""",2022,30978.0,171.467585,171.940174,170.958298,171.446213,115138.105218


In [ ]:
%%time
# let's try using .group_by_dynamic
(stocks_lazy
 .set_sorted('datetime')
 .group_by_dynamic(index_column='datetime', every='1y', group_by='ticker')
 .agg(cs.numeric().mean())
 .collect(engine='gpu')
 )

CPU times: user 6.48 s, sys: 3.09 s, total: 9.57 s
Wall time: 1.28 s


ticker,datetime,__index_level_0__,open,high,low,close,volume
str,datetime[ms],f64,f64,f64,f64,f64,f64
"""A""",1999-01-01 00:00:00,189.0,47.51504,47.881187,47.208522,47.554354,275892.612137
"""A""",2000-01-01 00:00:00,2011.5,69.742293,70.320141,69.139541,69.718236,222762.461727
"""A""",2001-01-01 00:00:00,5247.0,33.358924,33.59634,33.139426,33.366602,170170.826833
"""A""",2002-01-01 00:00:00,8477.5,22.984785,23.118953,22.851465,22.981314,189420.178133
"""A""",2003-01-01 00:00:00,11733.5,20.174097,20.250863,20.0986,20.177482,156302.885135
…,…,…,…,…,…,…,…
"""ZTS""",2020-01-01 00:00:00,24192.5,144.723845,145.203939,144.254515,144.727793,92883.735671
"""ZTS""",2021-01-01 00:00:00,27528.5,189.112176,189.522102,188.718892,189.122105,76024.96138
"""ZTS""",2022-01-01 00:00:00,30978.0,171.467585,171.940174,170.958298,171.446213,115138.105218


In [ ]:
%%time
# what's going on?
(stocks_lazy
 .set_sorted('datetime')
 .group_by_dynamic(index_column='datetime', every='1y', group_by='ticker')
 .agg(cs.numeric().mean())
 .collect(engine=pl.GPUEngine(device=0, raise_on_fail=True))
 )

ComputeError: 'cuda' conversion failed: NotImplementedError: ('Query execution with GPU not possible: unsupported operations.\nThe errors were:\n- NotImplementedError: dynamic group by', [NotImplementedError('dynamic group by')])

## Comparing Two Stocks

In [ ]:
%%time
# monthly results
(stocks_lazy
 .with_columns(year=pl.col('datetime').dt.year(),
               month=pl.col('datetime').dt.month())
 .group_by(['ticker','year', 'month'])
 .agg(cs.numeric().max())
 .sort(['ticker', 'year', 'month'])
 .collect(engine='gpu')
 )

CPU times: user 412 ms, sys: 128 ms, total: 540 ms
Wall time: 321 ms


ticker,year,month,__index_level_0__,open,high,low,close,volume
str,i32,i8,i64,f64,f64,f64,f64,i64
"""A""",1999,11,97,46.56,50.0,45.82,46.57,9275000
"""A""",1999,12,378,79.5,80.0,78.0,78.94,1393300
"""A""",2000,1,638,78.75,78.94,73.0,73.75,549700
"""A""",2000,2,898,114.0,115.44,111.25,114.25,730500
"""A""",2000,3,1197,156.0,161.0,149.56,158.0,1179900
…,…,…,…,…,…,…,…,…
"""ZTS""",2023,11,35938,181.09,181.39,180.91,181.07,1393038
"""ZTS""",2023,12,36219,201.13,201.92,200.65,201.06,1470827
"""ZTS""",2024,1,36512,199.29,199.83,198.89,199.37,850382


In [ ]:
%%time
# pivot so columns are close values
(stocks_lazy
 .with_columns(year=pl.col('datetime').dt.year(),
               month=pl.col('datetime').dt.month())
 .group_by(['ticker','year', 'month'])
 .agg(cs.numeric().max())
 .with_columns(date=pl.date('year', 'month', day=1))
 .collect(engine='gpu')
 .pivot(index='date', values='close', on='ticker')
 .sort('date')
 )

CPU times: user 12.1 s, sys: 4.85 s, total: 16.9 s
Wall time: 2.44 s


date,MRVL,ADSK,PEP,ANSS,DXCM,FAST,PSA,WAT,ROP,PH,MRK,EL,NUE,IP,EFX,POOL,JBHT,PGR,TFX,ON,NXPI,ABT,CARR,CL,ECL,SRE,UDR,CMCSA,KR,JPM,MET,TSN,UNP,HCA,GWW,INTU,…,ZTS,BR,PYPL,BBY,DLR,KMI,EPAM,ETSY,ELV,FOX,VRSN,WTW,HPE,LYB,DASH,WRK,BK,CTLT,FI,KDP,TEAM,DDOG,VTRS,BBWI,VMC,WBD,DD,LW,CCEP,ABNB,GFS,EG,UBER,KVUE,BALL,GEHC,CTRA
date,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1998-01-01,null,39.63,37.69,8.88,null,45.01,33.44,48.13,30.31,48.88,118.62,55.75,49.13,47.63,35.56,21.0,23.75,121.19,37.94,null,null,72.0,null,74.81,56.5,null,14.81,33.19,39.69,113.23,null,20.82,63.38,null,99.0,40.88,…,null,null,null,50.76,null,null,null,null,null,null,25.63,null,null,null,null,null,59.19,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
1998-02-01,null,49.82,36.88,9.44,null,48.5,33.63,50.13,33.44,46.75,130.5,59.88,51.88,48.94,35.94,23.5,27.75,117.38,41.56,null,null,75.69,null,81.63,29.0,null,14.38,34.81,43.38,124.74,null,19.51,63.69,null,99.88,53.26,…,null,null,null,60.94,null,null,null,null,null,null,33.87,null,null,null,null,null,58.63,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
1998-03-01,null,48.5,43.56,10.88,null,47.0,31.0,51.88,30.0,52.56,132.69,68.56,57.25,52.44,37.5,23.25,30.25,134.88,44.38,null,null,78.87,null,88.37,29.44,null,14.56,37.13,47.13,138.71,null,20.69,56.31,null,103.63,49.75,…,null,null,null,69.19,null,null,null,null,null,null,46.25,null,null,null,null,null,64.63,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
1998-04-01,null,47.19,44.69,12.0,null,56.25,32.63,55.0,31.76,52.31,132.63,73.75,60.38,55.12,40.69,24.0,33.5,138.76,46.38,null,null,78.63,null,91.25,31.94,null,14.5,37.63,47.25,147.79,null,20.5,58.19,null,109.44,54.75,…,null,null,null,75.81,null,null,null,null,null,null,48.0,null,null,null,null,null,66.51,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
1998-05-01,null,48.88,41.31,11.63,null,56.88,31.13,59.94,33.94,45.25,121.25,68.94,60.25,55.13,39.75,25.38,32.5,147.13,43.5,null,null,76.56,null,90.69,33.0,null,14.06,37.13,43.25,144.82,null,21.75,58.0,null,108.94,54.44,…,null,null,null,71.44,null,null,null,null,null,null,41.13,null,null,null,null,null,62.19,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2023-11-01,57.97,220.21,169.59,303.23,116.36,61.13,262.04,281.98,null,435.3,105.11,127.69,169.98,36.98,217.81,360.2,185.69,164.43,226.27,73.06,207.36,104.51,54.14,78.79,191.81,74.33,34.19,43.34,45.94,156.12,64.01,48.74,225.36,253.12,815.19,587.44,…,181.07,193.83,59.52,71.56,138.88,17.58,262.97,78.49,480.68,29.68,215.39,246.52,17.03,98.34,null,41.5,48.32,40.51,null,null,199.22,118.24,9.48,33.29,218.44,11.84,72.01,100.07,null,131.2,58.6,416.42,56.94,20.44,55.4,73.73,28.18
2023-12-01,61.7,245.74,172.03,363.67,125.93,65.63,307.83,335.4,null,463.15,109.2,149.62,179.23,38.09,251.97,404.84,207.3,165.13,256.95,86.65,237.97,110.55,59.11,79.74,200.87,77.29,38.77,44.93,45.72,170.61,67.58,54.07,246.45,276.35,840.26,631.06,…,201.06,206.61,63.79,78.72,139.24,17.88,302.11,89.37,488.27,28.36,220.37,249.24,17.28,97.16,null,43.49,52.26,45.8,null,null,247.55,124.4,10.87,43.72,227.14,12.61,77.18,108.72,null,149.77,62.59,411.96,63.41,22.13,60.55,78.27,26.53
2024-01-01,73.19,257.33,173.68,357.79,131.28,69.94,311.88,333.59,null,479.0,121.95,145.47,189.73,37.86,250.22,400.2,207.7,179.93,253.15,83.3,224.55,115.32,57.25,84.58,201.79,76.97,38.76,46.9,46.92,176.95,70.88,55.97,247.26,308.89,917.09,653.69,…,1

In [ ]:
%%time
# plot AMZN vs AAPL
(stocks_lazy
 .filter((pl.col('ticker') == 'AMZN') | (pl.col('ticker') == 'AAPL'))
 .with_columns(year=pl.col('datetime').dt.year(),
               month=pl.col('datetime').dt.month())
 .group_by(['ticker','year', 'month'])
 .agg(cs.numeric().max())
 .sort(['ticker', 'year', 'month'])
 .collect(engine='gpu')
 .with_columns(date=pl.date('year', 'month', day=1))
 # pivot not needed for altair
 # .pivot(index='date', values='close', on='ticker')
 .plot.line(x='date', y='close', color='ticker')
 )

CPU times: user 384 ms, sys: 110 ms, total: 494 ms
Wall time: 282 ms


alt.Chart(...)

## Conclusion

- Easy to install
- Works with existing Polars code
- Up to 13x speed improvement

Connect with me on social media and check out *Effective Polars* for getting started with the library.


# END

In [ ]:
with pl.Config() as cfg:
  cfg.set_verbose(True)
  res = (stocks_lazy
  .filter((pl.col('ticker') == 'MSFT') | (pl.col('ticker') == 'AAPL'))
  .with_columns(datetime=pl.col('datetime').dt.replace_time_zone('America/New_York'))
  .set_sorted('datetime')
  .group_by_dynamic(index_column='datetime', every='7d', period='5d', start_by='monday',
                    group_by='ticker')
  .agg(pl.col(pl.Float64).mean(),
        open_range=pl.col('open').max() - pl.col('open').min())
  .collect(engine='gpu')
  #.pivot(on='ticker', index='datetime')
  # altair doesn't require pivot because the 'color' param does that for us
  #.plot.line(x='datetime', y='close', color='ticker')

 )

In [ ]:
res

alt.Chart(...)

Bugs

In [ ]:

# This works on CPU
(stocks_lazy
#  .filter((pl.col('ticker') == 'AMZN') | (pl.col('ticker') == 'AAPL'))
 .with_columns(year=pl.col('datetime').dt.year(),
               month=pl.col('datetime').dt.month())
 .group_by(['ticker','year', 'month'])
 .agg(
     open=pl.col('open').first(),
      #close=pl.col('close').last(),
      high=pl.col('high').max(),
      #low=pl.col('low').min(),
      #volume=pl.col('volume').sum()
      )
 .sort(['ticker', 'year', 'month'])
 .collect(engine='gpu')
 )

ComputeError: KeyError: '__index_level_0__'

## Normal Install

In [ ]:
!pip install polars[gpu] --extra-index-url=https://pypi.nvidia.com

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 99.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 113.4 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3070, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2863, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command

In [ ]:

GOOGLE_COLAB=True

if GOOGLE_COLAB:
    from google.colab import drive
    drive.mount("/content/drive")

    !pip install polars==1.5
    !pip install "/content/drive/MyDrive/cudf_polars_cu12-24.8.0a433-py3-none-any.whl" --extra-index-url=https://pypi.anaconda.org/rapidsai-wheels-nightly/simple

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://pypi.anaconda.org/rapidsai-wheels-nightly/simple
Processing ./drive/MyDrive/cudf_polars_cu12-24.8.0a433-py3-none-any.whl
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 6.9 MB/s eta 0:00:00
  Created wheel for cudf-cu12: filename=cudf_cu12-24.8.2-cp310-cp310-manylinux_2_28_x86_64.whl size=516733978 sha256=d93d3c6985d5ef180c68482915a14a0f2503f1d5c1b3c23333e56b9de28295a8
  Stored in directory: /root/.cache/pip/wheels/2d/4d/cf/43886910338252088abcdcd8bf06472af8d678c6f9bcb4c5cd
  Created wheel for rmm-cu12: filename=rmm_cu12-24.8.2-cp310

# Load Library

In [ ]:
import polars as pl
pl.__version__

'1.7.1'

## Rainfall Data

In [ ]:
!wget https://storage.googleapis.com/rapidsai/polars-demo/rainfall_data_2010_2020.csv

--2024-10-22 15:41:35--  https://storage.googleapis.com/rapidsai/polars-demo/rainfall_data_2010_2020.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.194.207, 142.251.12.207, 172.253.118.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.194.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 131421 (128K) [text/csv]
Saving to: ‘rainfall_data_2010_2020.csv’

rainfall_data_2010_ 100%[===================>] 128.34K   291KB/s    in 0.4s    

2024-10-22 15:41:37 (291 KB/s) - ‘rainfall_data_2010_2020.csv’ saved [131421/131421]



In [ ]:
rain_raw = pl.scan_csv("rainfall_data_2010_2020.csv")

In [ ]:
rain_raw.head(5).collect(engine='gpu')

Location,Rainfall (inches),Date,YEAR,MONTH,DAY
str,f64,i64,i64,i64,i64
"""Tatooine""",0.33,20100101,2010,1,1
"""Tatooine""",0.0,20100102,2010,1,2
"""Tatooine""",0.28,20100103,2010,1,3
"""Tatooine""",0.26,20100104,2010,1,4
"""Tatooine""",0.39,20100105,2010,1,5


In [ ]:
rain = (rain_raw
        .with_columns(pl.col('Date').cast(pl.Utf8).str.strptime(pl.Date, '%Y%m%d'))
        .collect(engine='gpu')
       )
rain

Location,Rainfall (inches),Date,YEAR,MONTH,DAY
str,f64,date,i64,i64,i64
"""Tatooine""",0.33,2010-01-01,2010,1,1
"""Tatooine""",0.0,2010-01-02,2010,1,2
"""Tatooine""",0.28,2010-01-03,2010,1,3
"""Tatooine""",0.26,2010-01-04,2010,1,4
"""Tatooine""",0.39,2010-01-05,2010,1,5
…,…,…,…,…,…
"""Tatooine""",0.0,2020-12-27,2020,12,27
"""Tatooine""",0.0,2020-12-28,2020,12,28
"""Tatooine""",0.07,2020-12-29,2020,12,29


## Transaction Data

In [ ]:
!wget https://storage.googleapis.com/rapidsai/polars-demo/transactions-t4-20.parquet -O transactions.parquetst



--2024-10-22 15:43:25--  https://storage.googleapis.com/rapidsai/polars-demo/transactions-t4-20.parquet
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.175.207, 74.125.24.207, 142.251.10.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.175.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 857658422 (818M) [application/octet-stream]
Saving to: ‘transactions.parquet’

transactions.parque 100%[===================>] 817.93M  17.6MB/s    in 48s     

2024-10-22 15:44:15 (17.1 MB/s) - ‘transactions.parquet’ saved [857658422/857658422]



In [ ]:
trans_raw = pl.scan_parquet("transactions.parquet")

## Adapted Time Series Content

In [ ]:
(trans_raw.head(5).collect(engine='gpu'))

CUST_ID,START_DATE,END_DATE,TRANS_ID,DATE,YEAR,MONTH,DAY,EXP_TYPE,AMOUNT
str,date,date,str,date,i64,i64,i64,str,f64
"""CI6XLYUMQK""",2015-05-01,null,"""T8I9ZB5A6X90UG8""",2015-09-11,2015,9,11,"""Motor/Travel""",20.27
"""CI6XLYUMQK""",2015-05-01,null,"""TZ4JSLS7SC7FO9H""",2017-02-08,2017,2,8,"""Motor/Travel""",12.85
"""CI6XLYUMQK""",2015-05-01,null,"""TTUKRDDJ6B6F42H""",2015-08-01,2015,8,1,"""Housing""",383.8
"""CI6XLYUMQK""",2015-05-01,null,"""TDUHFRUKGPPI6HD""",2019-03-16,2019,3,16,"""Entertainment""",5.72
"""CI6XLYUMQK""",2015-05-01,null,"""T0JBZHBMSVRFMMD""",2015-05-15,2015,5,15,"""Entertainment""",11.06


In [ ]:
# Convert YEAR, MONTH, DAY to a date
(trans_raw
 .with_columns(T_DATETIME=pl.datetime('YEAR', 'MONTH', 'DAY'),
               T_DATE=pl.date('YEAR', 'MONTH', 'DAY'))
 .head(5).collect(engine='gpu')
 )

CUST_ID,START_DATE,END_DATE,TRANS_ID,DATE,YEAR,MONTH,DAY,EXP_TYPE,AMOUNT,T_DATETIME,T_DATE
str,date,date,str,date,i64,i64,i64,str,f64,datetime[μs],date
"""CI6XLYUMQK""",2015-05-01,null,"""T8I9ZB5A6X90UG8""",2015-09-11,2015,9,11,"""Motor/Travel""",20.27,2015-09-11 00:00:00,2015-09-11
"""CI6XLYUMQK""",2015-05-01,null,"""TZ4JSLS7SC7FO9H""",2017-02-08,2017,2,8,"""Motor/Travel""",12.85,2017-02-08 00:00:00,2017-02-08
"""CI6XLYUMQK""",2015-05-01,null,"""TTUKRDDJ6B6F42H""",2015-08-01,2015,8,1,"""Housing""",383.8,2015-08-01 00:00:00,2015-08-01
"""CI6XLYUMQK""",2015-05-01,null,"""TDUHFRUKGPPI6HD""",2019-03-16,2019,3,16,"""Entertainment""",5.72,2019-03-16 00:00:00,2019-03-16
"""CI6XLYUMQK""",2015-05-01,null,"""T0JBZHBMSVRFMMD""",2015-05-15,2015,5,15,"""Entertainment""",11.06,2015-05-15 00:00:00,2015-05-15


In [ ]:
#change time zone
# Convert YEAR, MONTH, DAY to a date
(trans_raw
 .with_columns(T_DATE=pl.datetime('YEAR', 'MONTH', 'DAY').dt.replace_time_zone('UTC').dt.convert_time_zone('America/Denver'))
 .head(5).collect(engine='gpu')
 )

CUST_ID,START_DATE,END_DATE,TRANS_ID,DATE,YEAR,MONTH,DAY,EXP_TYPE,AMOUNT,T_DATE
str,date,date,str,date,i64,i64,i64,str,f64,"datetime[μs, America/Denver]"
"""CI6XLYUMQK""",2015-05-01,null,"""T8I9ZB5A6X90UG8""",2015-09-11,2015,9,11,"""Motor/Travel""",20.27,2015-09-10 18:00:00 MDT
"""CI6XLYUMQK""",2015-05-01,null,"""TZ4JSLS7SC7FO9H""",2017-02-08,2017,2,8,"""Motor/Travel""",12.85,2017-02-07 17:00:00 MST
"""CI6XLYUMQK""",2015-05-01,null,"""TTUKRDDJ6B6F42H""",2015-08-01,2015,8,1,"""Housing""",383.8,2015-07-31 18:00:00 MDT
"""CI6XLYUMQK""",2015-05-01,null,"""TDUHFRUKGPPI6HD""",2019-03-16,2019,3,16,"""Entertainment""",5.72,2019-03-15 18:00:00 MDT
"""CI6XLYUMQK""",2015-05-01,null,"""T0JBZHBMSVRFMMD""",2015-05-15,2015,5,15,"""Entertainment""",11.06,2015-05-14 18:00:00 MDT


In [ ]:
# time aggregations
(trans_raw
 .with_columns(G_YEAR=pl.col('DATE').dt.year())
 .group_by('G_YEAR')
 .agg(pl.col(pl.Float64).mean())
 .collect(engine='gpu')
)

G_YEAR,AMOUNT
i32,f64
2016,66.621701
2020,88.060301
2012,52.326643
2010,48.671244
2013,55.230212
…,…
2015,62.759225
2011,49.592248
2018,76.215547


In [ ]:
# time aggregations
# fix order
(trans_raw
 .with_columns(G_YEAR=pl.col('DATE').dt.year())
 .group_by('G_YEAR', maintain_order=True)
 .agg(pl.col(pl.Float64).mean())
 .collect(engine='gpu')
)

G_YEAR,AMOUNT
i32,f64
2015,62.759225
2017,71.003062
2019,81.838656
2016,66.621701
2018,76.215547
…,…
2014,58.924756
2012,52.326643
2010,48.671244


In [ ]:
# time aggregations
# fix order
(trans_raw

 .with_columns(G_YEAR=pl.col('DATE').dt.year())
 .sort('G_YEAR')
 .group_by('G_YEAR', maintain_order=True)
 .agg(pl.col(pl.Float64).mean())
 .collect(engine='gpu')
)

G_YEAR,AMOUNT
i32,f64
2010,48.671244
2011,49.592248
2012,52.326643
2013,55.230212
2014,58.924756
…,…
2016,66.621701
2017,71.003062
2018,76.215547


In [ ]:
# 2 month intervals
(trans_raw
 #.with_columns(G_YEAR=pl.col('DATE').dt.year())
 .sort('DATE')
 .group_by_dynamic(index_column='DATE', every='2mo')
 .agg(pl.col(pl.Float64).mean())
 .collect(engine='gpu')
)

DATE,AMOUNT
date,f64
2010-01-01,44.308244
2010-03-01,45.912862
2010-05-01,46.337452
2010-07-01,46.402012
2010-09-01,47.685412
…,…
2020-03-01,83.434127
2020-05-01,84.461499
2020-07-01,85.520126


In [ ]:
# weekdays
# 2 month intervals
(trans_raw
 #.with_columns(G_YEAR=pl.col('DATE').dt.year())
 .sort('DATE')
 .group_by_dynamic(index_column='DATE', every='7d', period='5d', start_by='monday')
 .agg(pl.col(pl.Float64).mean())
 .collect(engine='gpu')
)

DATE,AMOUNT
date,f64
2009-12-28,77.272055
2010-01-04,65.05641
2010-01-11,28.240572
2010-01-18,29.652597
2010-01-25,28.137497
…,…
2020-11-23,74.679038
2020-11-30,147.204881
2020-12-07,135.613337


In [ ]:

# add year mean
(trans_raw
 .sort('DATE')
 .with_columns(YEAR_MEAN=pl.col('AMOUNT').mean().over('YEAR'),
               YEAR_TOTAL=pl.col('AMOUNT').sum().over('YEAR'))
 .with_columns(YEAR_PCT=(pl.col('AMOUNT')/pl.col('YEAR_TOTAL'))*100)
 .collect(engine='gpu')
)

CPU times: user 31.4 s, sys: 8.63 s, total: 40.1 s
Wall time: 9.45 s


CUST_ID,START_DATE,END_DATE,TRANS_ID,DATE,YEAR,MONTH,DAY,EXP_TYPE,AMOUNT,YEAR_MEAN,YEAR_TOTAL,YEAR_PCT
str,date,date,str,date,i64,i64,i64,str,f64,f64,f64,f64
"""CVPBSTACKK""",2010-01-01,2011-02-01,"""TLQWD9MHZ1TQJFG""",2010-01-01,2010,1,1,"""Health""",7.17,48.671244,3.5308e7,0.00002
"""CVPBSTACKK""",2010-01-01,2011-02-01,"""TT6F3O8VQK4KD8X""",2010-01-01,2010,1,1,"""Housing""",772.92,48.671244,3.5308e7,0.002189
"""CVPBSTACKK""",2010-01-01,2011-02-01,"""TV3VLJ5KACDTBVB""",2010-01-01,2010,1,1,"""Entertainment""",14.73,48.671244,3.5308e7,0.000042
"""CVPBSTACKK""",2010-01-01,2011-02-01,"""TJN5PDZA03INVUN""",2010-01-01,2010,1,1,"""Entertainment""",10.23,48.671244,3.5308e7,0.000029
"""CVPBSTACKK""",2010-01-01,2011-02-01,"""T96NB83Z1MBM21T""",2010-01-01,2010,1,1,"""Entertainment""",11.19,48.671244,3.5308e7,0.000032
…,…,…,…,…,…,…,…,…,…,…,…,…
"""CFM0ER1QC4""",2019-04-01,null,"""T2USUEUD12JWGEO""",2020-12-27,2020,12,27,"""Entertainment""",23.33,88.060301,5.3188e8,0.000004
"""CDM6EPDW8A""",2017-07-01,null,"""T4R3JMLA6I8V77Z""",2020-12-27,2020,12,27,"""Entertainment""",53.19,88.060301,5.3188e8,0.00001
"""CDM6EPDW8A""",2017-07-01,null,"""TIQQQEH0E45THQ9""",2020-12-27,2020,12,27,"""Entertainment""",81.02,88.060301,5.3188e8,0.000015


In [ ]:
col = pl.col('f')
dir(col.dt)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_accessor',
 '_pyexpr',
 'add_business_days',
 'base_utc_offset',
 'cast_time_unit',
 'century',
 'combine',
 'convert_time_zone',
 'date',
 'datetime',
 'day',
 'dst_offset',
 'epoch',
 'hour',
 'is_leap_year',
 'iso_year',
 'microsecond',
 'millennium',
 'millisecond',
 'minute',
 'month',
 'month_end',
 'month_start',
 'nanosecond',
 'offset_by',
 'ordinal_day',
 'quarter',
 'replace_time_zone',
 'round',
 'second',
 'strftime',
 'time',
 'timestamp',
 'to_string',
 'total_days',
 'total_hours',
 'total_microseconds',
 'total_milliseconds',
 'total_minutes',
 'total_nanoseconds',
 'total_seconds',
 'truncate',
 'week'

In [ ]:
col.dt.truncate?

In [ ]:
# add year mean
(trans_raw
 .sort('DATE')
 .with_columns(QUARTER=pl.col('DATE').dt.truncate('1q'))
 .with_columns(YEAR_MEAN=pl.col('AMOUNT').mean().over('YEAR'),
               YEAR_TOTAL=pl.col('AMOUNT').sum().over('YEAR'),
               QUARTER_MEAN=pl.col('AMOUNT').mean().over('QUARTER'),
               QUARTER_TOTAL=pl.col('AMOUNT').sum().over('QUARTER'))
 .with_columns(QUARTER_PCT=(pl.col('AMOUNT')/pl.col('QUARTER_TOTAL'))*100)
 .collect(engine='gpu')
)

CUST_ID,START_DATE,END_DATE,TRANS_ID,DATE,YEAR,MONTH,DAY,EXP_TYPE,AMOUNT,QUARTER,YEAR_MEAN,YEAR_TOTAL,QUARTER_MEAN,QUARTER_TOTAL,QUARTER_PCT
str,date,date,str,date,i64,i64,i64,str,f64,date,f64,f64,f64,f64,f64
"""CVPBSTACKK""",2010-01-01,2011-02-01,"""TLQWD9MHZ1TQJFG""",2010-01-01,2010,1,1,"""Health""",7.17,2010-01-01,48.671244,3.5308e7,45.047198,2.4914e6,0.000288
"""CVPBSTACKK""",2010-01-01,2011-02-01,"""TT6F3O8VQK4KD8X""",2010-01-01,2010,1,1,"""Housing""",772.92,2010-01-01,48.671244,3.5308e7,45.047198,2.4914e6,0.031023
"""CVPBSTACKK""",2010-01-01,2011-02-01,"""TV3VLJ5KACDTBVB""",2010-01-01,2010,1,1,"""Entertainment""",14.73,2010-01-01,48.671244,3.5308e7,45.047198,2.4914e6,0.000591
"""CVPBSTACKK""",2010-01-01,2011-02-01,"""TJN5PDZA03INVUN""",2010-01-01,2010,1,1,"""Entertainment""",10.23,2010-01-01,48.671244,3.5308e7,45.047198,2.4914e6,0.000411
"""CVPBSTACKK""",2010-01-01,2011-02-01,"""T96NB83Z1MBM21T""",2010-01-01,2010,1,1,"""Entertainment""",11.19,2010-01-01,48.671244,3.5308e7,45.047198,2.4914e6,0.000449
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""CFM0ER1QC4""",2019-04-01,null,"""T2USUEUD12JWGEO""",2020-12-27,2020,12,27,"""Entertainment""",23.33,2020-10-01,88.060301,5.3188e8,100.914633,1.4041e8,0.000017
"""CDM6EPDW8A""",2017-07-01,null,"""T4R3JMLA6I8V77Z""",2020-12-27,2020,12,27,"""Entertainment""",53.19,2020-10-01,88.060301,5.3188e8,100.914633,1.4041e8,0.000038
"""CDM6EPDW8A""",2017-07-01,null,"""TIQQQEH0E45THQ9""",2020-12-27,2020,12,27,"""Entertainment""",81.02,2020-10-01,88.060301,5.3188e8,100.914633,1.4041e8,0.000058


In [ ]:
# BUG!
#!wget https://storage.googleapis.com/rapidsai/polars-demo/transactions-t4-20.parquet -O transactions.parquet

#trans_raw = pl.scan_parquet("transactions.parquet")
# 2 month intervals with exp_type
(trans_raw
 .sort('DATE')
 .group_by_dynamic(index_column='DATE', every='2mo', group_by='EXP_TYPE')
 .agg(pl.col(pl.Float64).mean())
 #.collect(engine='gpu')
 .collect()
)

EXP_TYPE,AMOUNT
str,f64
"""Health""",72.9537
"""Housing""",1172.261043
"""Entertainment""",34.161607
"""Groceries""",33.706856
"""Motor/Travel""",134.374954
…,…
"""Savings""",117.850622
"""Education""",183.861958
"""Tax""",234.786829


In [ ]:
# 2 month intervals with exp_type
(trans_raw
 .sort('DATE')
 .group_by([pl.col('DATE').dt.truncate('2mo'), 'EXP_TYPE'])
 #.group_by_dynamic(index_column='DATE', every='2mo', group_by='EXP_TYPE')
 .agg(pl.col(pl.Float64).mean())
 .collect(engine='gpu')
 #.collect()
)

DATE,EXP_TYPE,AMOUNT
date,str,f64
2014-07-01,"""Housing""",1056.059456
2018-07-01,"""Education""",191.531353
2018-11-01,"""Gambling""",108.840434
2013-05-01,"""Savings""",113.670657
2016-01-01,"""Education""",182.417347
…,…,…
2015-11-01,"""Entertainment""",40.080053
2016-07-01,"""Motor/Travel""",112.136303
2011-09-01,"""Fines""",129.364595


In [ ]:
# Pivoiting results
(trans_raw
 .sort('DATE')
 .group_by([pl.col('DATE').dt.truncate('2mo'), 'EXP_TYPE'])
 #.group_by_dynamic(index_column='DATE', every='2mo', group_by='EXP_TYPE')
 .agg(pl.col(pl.Float64).mean())
 .pivot(index='DATE', on='EXP_TYPE')
 .collect(engine='gpu')
 #.collect()
)

AttributeError: 'LazyFrame' object has no attribute 'pivot'

In [ ]:
# Pivoting results
(trans_raw
 .sort('DATE')
 .group_by([pl.col('DATE').dt.truncate('1mo'), 'EXP_TYPE'])
 #.group_by_dynamic(index_column='DATE', every='2mo', group_by='EXP_TYPE')
 .agg(pl.col(pl.Float64).mean())
 .collect(engine='gpu')
 .pivot(index='DATE', on='EXP_TYPE')
 .sort('DATE')
 #.collect()
)

DATE,Health,Fines,Housing,Entertainment,Education,Gambling,Motor/Travel,Bills and Utilities,Savings,Groceries,Clothing,Tax
date,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2010-01-01,49.243985,105.066667,844.379063,27.554687,153.749925,55.93197,32.32889,85.415487,123.625532,18.254345,122.499091,216.10959
2010-02-01,52.40485,91.3675,854.132966,27.988736,160.649014,60.101948,32.789203,90.495608,123.314031,18.177861,137.002939,230.370124
2010-03-01,52.340652,150.291,856.182135,28.793104,159.329442,59.184588,34.729718,94.857936,113.273345,19.182945,141.960239,232.961967
2010-04-01,57.05242,140.866364,870.464208,28.160584,166.454911,60.991605,35.770638,95.238054,119.252036,19.804399,134.370789,238.061697
2010-05-01,55.943761,140.708,868.06131,28.896759,163.521686,57.826325,35.936447,94.464738,126.245885,19.126746,142.758522,234.924417
…,…,…,…,…,…,…,…,…,…,…,…,…
2020-08-01,88.495778,149.150962,1399.646157,34.768955,198.576702,92.498331,235.799757,115.387243,118.578675,42.485611,304.745473,233.777692
2020-09-01,88.814638,133.962159,1408.80332,35.443963,199.708996,94.472934,237.636033,115.432807,118.19381,42.64545,312.099097,234.608419
2020-10-01,88.715024,136.703247,1415.498122,38.977062,200.386079,104.517923,267.082973,115.944788,118.912163,47.013118,346.404144,234.351233


In [ ]:
import hvplot.polars
# Plotting
(trans_raw
 .sort('DATE')
 .group_by([pl.col('DATE').dt.truncate('1mo'), 'EXP_TYPE'])
 #.group_by_dynamic(index_column='DATE', every='2mo', group_by='EXP_TYPE')
 .agg(pl.col(pl.Float64).mean())
 .collect(engine='gpu')
 .pivot(index='DATE', on='EXP_TYPE')
 .sort('DATE')
 #.collect()
 .hvplot.line(x='DATE', y=['Clothing', 'Groceries', 'Motor/Travel'])
)

:NdOverlay   [Variable]
   :Curve   [DATE]   (value)

In [ ]:
# join with rain data
(trans_raw
 .join(rain.lazy(), left_on='DATE', right_on='Date', how='left')
 .sort('DATE')
 .group_by([pl.col('DATE').dt.truncate('1mo'), 'EXP_TYPE'])
 ##.group_by_dynamic(index_column='DATE', every='2mo', group_by='EXP_TYPE')
 .agg(pl.col(pl.Float64).mean())
 .collect(engine='gpu')
 #.pivot(index='DATE', on=['EXP_TYPE', 'Rainfall (inches)'])
 #.sort('DATE')
 ##.collect()
 #.hvplot.line(x='DATE', y=['Clothing', 'Groceries', 'Motor/Travel'])
 )

DATE,EXP_TYPE,AMOUNT,Rainfall (inches)
date,str,f64,f64
2010-01-01,"""Groceries""",18.254345,0.20395
2018-02-01,"""Housing""",1248.100206,0.208469
2016-02-01,"""Savings""",121.444292,0.175837
2015-06-01,"""Education""",179.279382,0.023974
2019-06-01,"""Tax""",235.590289,0.035853
…,…,…,…
2020-08-01,"""Tax""",233.777692,0.016399
2015-05-01,"""Motor/Travel""",90.886005,0.156317
2011-07-01,"""Housing""",898.910996,0.050838


## Time Series from Effective Polars

In [ ]:
>>> import urllib.request

>>> def download_and_modify_url(url, local_filename):
...     urllib.request.urlretrieve(url, local_filename)
...     with open(local_filename, 'r') as file:
...         lines = file.readlines()
...
...     with open(local_filename, 'w') as file:
...         for i, line in enumerate(lines):
...             if i <34 or i == 35:
...                 continue
...             file.write(line)

>>> url = 'https://github.com/mattharrison/datasets/raw/master'\
...     '/data/dirtydevil.txt'
>>> local_filename = 'devilclean.txt'
>>> download_and_modify_url(url, local_filename)


In [ ]:
>>> import polars as pl

>>> def tweak_river(df_):
...     return (df_
...         .select('agency_cd', 'site_no', 'tz_cd',
...                 pl.col('datetime').str.to_datetime(),
...                 cfs=pl.col('144166_00060'),
...                 gage_height=pl.col('144167_00065').cast(pl.Float64)
...                 )
...             )

>>> raw = pl.scan_csv('devilclean.txt', separator='\t')
>>> dd = tweak_river(raw).collect(engine='gpu')
>>> print(dd)

shape: (539_305, 6)
┌───────────┬─────────┬───────┬─────────────────────┬──────┬─────────────┐
│ agency_cd ┆ site_no ┆ tz_cd ┆ datetime            ┆ cfs  ┆ gage_height │
│ ---       ┆ ---     ┆ ---   ┆ ---                 ┆ ---  ┆ ---         │
│ str       ┆ i64     ┆ str   ┆ datetime[μs]        ┆ f64  ┆ f64         │
╞═══════════╪═════════╪═══════╪═════════════════════╪══════╪═════════════╡
│ USGS      ┆ 9333500 ┆ MDT   ┆ 2001-05-07 01:00:00 ┆ 71.0 ┆ null        │
│ USGS      ┆ 9333500 ┆ MDT   ┆ 2001-05-07 01:15:00 ┆ 71.0 ┆ null        │
│ USGS      ┆ 9333500 ┆ MDT   ┆ 2001-05-07 01:30:00 ┆ 71.0 ┆ null        │
│ USGS      ┆ 9333500 ┆ MDT   ┆ 2001-05-07 01:45:00 ┆ 70.0 ┆ null        │
│ USGS      ┆ 9333500 ┆ MDT   ┆ 2001-05-07 02:00:00 ┆ 70.0 ┆ null        │
│ …         ┆ …       ┆ …     ┆ …                   ┆ …    ┆ …           │
│ USGS      ┆ 9333500 ┆ MDT   ┆ 2020-09-28 08:30:00 ┆ 9.53 ┆ 6.16        │
│ USGS      ┆ 9333500 ┆ MDT   ┆ 2020-09-28 08:45:00 ┆ 9.2  ┆ 6.15        │
│ USG

In [ ]:
# Converting Dates
>>> print(pl.scan_csv('devilclean.txt', separator='\t',
...                   try_parse_dates=True).collect(engine='gpu'))

shape: (539_305, 8)
┌───────────┬─────────┬─────────────┬───────┬─────────────┬─────────────┬─────────────┬────────────┐
│ agency_cd ┆ site_no ┆ datetime    ┆ tz_cd ┆ 144166_0006 ┆ 144166_0006 ┆ 144167_0006 ┆ 144167_000 │
│ ---       ┆ ---     ┆ ---         ┆ ---   ┆ 0           ┆ 0_cd        ┆ 5           ┆ 65_cd      │
│ str       ┆ i64     ┆ datetime[μs ┆ str   ┆ ---         ┆ ---         ┆ ---         ┆ ---        │
│           ┆         ┆ ]           ┆       ┆ f64         ┆ str         ┆ str         ┆ str        │
╞═══════════╪═════════╪═════════════╪═══════╪═════════════╪═════════════╪═════════════╪════════════╡
│ USGS      ┆ 9333500 ┆ 2001-05-07  ┆ MDT   ┆ 71.0        ┆ A:[91]      ┆ null        ┆ null       │
│           ┆         ┆ 01:00:00    ┆       ┆             ┆             ┆             ┆            │
│ USGS      ┆ 9333500 ┆ 2001-05-07  ┆ MDT   ┆ 71.0        ┆ A:[91]      ┆ null        ┆ null       │
│           ┆         ┆ 01:15:00    ┆       ┆             ┆            

In [ ]:
>>> format = '%Y-%m-%d %H:%M'
>>> print(raw
... .select(original=pl.col('datetime'),
...         to_datetime=pl.col('datetime').str.to_datetime(format),
...         to_date=pl.col('datetime').str.to_date(format),
...         strptime=pl.col('datetime').str.strptime(pl.Datetime, format),
...         # line below fails
...         #cast=pl.col('datetime').cast(pl.Datetime)
...         )
... .collect(engine='gpu')
... )

shape: (539_305, 4)
┌──────────────────┬─────────────────────┬────────────┬─────────────────────┐
│ original         ┆ to_datetime         ┆ to_date    ┆ strptime            │
│ ---              ┆ ---                 ┆ ---        ┆ ---                 │
│ str              ┆ datetime[μs]        ┆ date       ┆ datetime[μs]        │
╞══════════════════╪═════════════════════╪════════════╪═════════════════════╡
│ 2001-05-07 01:00 ┆ 2001-05-07 01:00:00 ┆ 2001-05-07 ┆ 2001-05-07 01:00:00 │
│ 2001-05-07 01:15 ┆ 2001-05-07 01:15:00 ┆ 2001-05-07 ┆ 2001-05-07 01:15:00 │
│ 2001-05-07 01:30 ┆ 2001-05-07 01:30:00 ┆ 2001-05-07 ┆ 2001-05-07 01:30:00 │
│ 2001-05-07 01:45 ┆ 2001-05-07 01:45:00 ┆ 2001-05-07 ┆ 2001-05-07 01:45:00 │
│ 2001-05-07 02:00 ┆ 2001-05-07 02:00:00 ┆ 2001-05-07 ┆ 2001-05-07 02:00:00 │
│ …                ┆ …                   ┆ …          ┆ …                   │
│ 2020-09-28 08:30 ┆ 2020-09-28 08:30:00 ┆ 2020-09-28 ┆ 2020-09-28 08:30:00 │
│ 2020-09-28 08:45 ┆ 2020-09-28 08:45:00 ┆ 2

In [ ]:
## Combining Columns
>>> print(raw
...  .select(to_datetime=pl.col('datetime').str.to_datetime(format))
...  .with_columns(month=pl.col('to_datetime').dt.strftime('%m'),
...                year=pl.col('to_datetime').dt.strftime('%Y'))
...  .collect(engine='gpu')
... )

shape: (539_305, 3)
┌─────────────────────┬───────┬──────┐
│ to_datetime         ┆ month ┆ year │
│ ---                 ┆ ---   ┆ ---  │
│ datetime[μs]        ┆ str   ┆ str  │
╞═════════════════════╪═══════╪══════╡
│ 2001-05-07 01:00:00 ┆ 05    ┆ 2001 │
│ 2001-05-07 01:15:00 ┆ 05    ┆ 2001 │
│ 2001-05-07 01:30:00 ┆ 05    ┆ 2001 │
│ 2001-05-07 01:45:00 ┆ 05    ┆ 2001 │
│ 2001-05-07 02:00:00 ┆ 05    ┆ 2001 │
│ …                   ┆ …     ┆ …    │
│ 2020-09-28 08:30:00 ┆ 09    ┆ 2020 │
│ 2020-09-28 08:45:00 ┆ 09    ┆ 2020 │
│ 2020-09-28 09:00:00 ┆ 09    ┆ 2020 │
│ 2020-09-28 09:15:00 ┆ 09    ┆ 2020 │
│ 2020-09-28 09:30:00 ┆ 09    ┆ 2020 │
└─────────────────────┴───────┴──────┘


In [ ]:
>>> print(raw
...  .select(to_datetime=pl.col('datetime').str.to_datetime(format))
...  .with_columns(month=pl.col('to_datetime').dt.strftime('%m'),
...                year=pl.col('to_datetime').dt.strftime('%Y'))
...  .select(pl.date(pl.col('year'), pl.col('month'), 1))
... )

shape: (539_305, 1)
┌────────────┐
│ date       │
│ ---        │
│ date       │
╞════════════╡
│ 2001-05-01 │
│ 2001-05-01 │
│ 2001-05-01 │
│ 2001-05-01 │
│ 2001-05-01 │
│ …          │
│ 2020-09-01 │
│ 2020-09-01 │
│ 2020-09-01 │
│ 2020-09-01 │
│ 2020-09-01 │
└────────────┘


In [ ]:
## Changing Timezones
>>> format = '%Y-%m-%d %H:%M'
>>> print(raw
... .select(original=pl.col('datetime'),
...         naive=pl.col('datetime').str.to_datetime(format),
...         utc=pl.col('datetime').str.to_datetime(format)
...             .dt.replace_time_zone('UTC'),
...         Denver=pl.col('datetime').str.to_datetime(format)
...             .dt.replace_time_zone('UTC')
...             .dt.convert_time_zone('America/Denver'),
...         Denver2=(pl.col('datetime') + ' ' + (pl.col('tz_cd')
...             .str.replace('MST', '-0700').str.replace('MDT', '-0600')))
...             .str.to_datetime('%Y-%m-%d %H:%M %z')
...             .dt.convert_time_zone('America/Denver'),
...         Denver3=(pl.col('datetime').str.to_datetime(format,
...                  time_zone='America/Denver', ambiguous='earliest'))
...         )
... .collect(engine='gpu')
... )


shape: (539_305, 6)
┌────────────┬─────────────────┬────────────────┬────────────────┬────────────────┬────────────────┐
│ original   ┆ naive           ┆ utc            ┆ Denver         ┆ Denver2        ┆ Denver3        │
│ ---        ┆ ---             ┆ ---            ┆ ---            ┆ ---            ┆ ---            │
│ str        ┆ datetime[μs]    ┆ datetime[μs,   ┆ datetime[μs,   ┆ datetime[μs,   ┆ datetime[μs,   │
│            ┆                 ┆ UTC]           ┆ America/Denver ┆ America/Denver ┆ America/Denver │
│            ┆                 ┆                ┆ ]              ┆ ]              ┆ ]              │
╞════════════╪═════════════════╪════════════════╪════════════════╪════════════════╪════════════════╡
│ 2001-05-07 ┆ 2001-05-07      ┆ 2001-05-07     ┆ 2001-05-06     ┆ 2001-05-07     ┆ 2001-05-07     │
│ 01:00      ┆ 01:00:00        ┆ 01:00:00 UTC   ┆ 19:00:00 MDT   ┆ 01:00:00 MDT   ┆ 01:00:00 MDT   │
│ 2001-05-07 ┆ 2001-05-07      ┆ 2001-05-07     ┆ 2001-05-06     ┆ 2001

In [ ]:
import polars as pl

def tweak_river(df_, cfs_col, gage_height_col):
    return (df_
        .select(
            'agency_cd', 'site_no',
            cfs=pl.col(cfs_col),
            gage_height=pl.col(gage_height_col).cast(pl.Float64),
            datetime=(pl.col('datetime') + ' ' + (pl.col('tz_cd')
                .str.replace('MST', '-0700').str.replace('MDT', '-0600')))
                .str.to_datetime('%Y-%m-%d %H:%M %z')
                .dt.convert_time_zone('America/Denver')
            )
        )

dd = tweak_river(raw, cfs_col='144166_00060',
                gage_height_col='144167_00065')
dd.collect(engine='gpu')

agency_cd,site_no,cfs,gage_height,datetime
str,i64,f64,f64,"datetime[μs, America/Denver]"
"""USGS""",9333500,71.0,null,2001-05-07 01:00:00 MDT
"""USGS""",9333500,71.0,null,2001-05-07 01:15:00 MDT
"""USGS""",9333500,71.0,null,2001-05-07 01:30:00 MDT
"""USGS""",9333500,70.0,null,2001-05-07 01:45:00 MDT
"""USGS""",9333500,70.0,null,2001-05-07 02:00:00 MDT
…,…,…,…,…
"""USGS""",9333500,9.53,6.16,2020-09-28 08:30:00 MDT
"""USGS""",9333500,9.2,6.15,2020-09-28 08:45:00 MDT
"""USGS""",9333500,9.2,6.15,2020-09-28 09:00:00 MDT


In [ ]:
%%timeit
dd = tweak_river(raw, cfs_col='144166_00060',
                gage_height_col='144167_00065').collect()

87.8 ms ± 6.89 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
%%timeit
dd = tweak_river(raw, cfs_col='144166_00060',
                gage_height_col='144167_00065').collect(engine='gpu')

91.7 ms ± 7.81 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
## Time Aggregations
>>> print(dd
...  .with_columns(year=pl.col('datetime').dt.year())
...  .group_by('year')
...  .agg(pl.col(pl.Float64).mean())
...  )

shape: (20, 3)
┌──────┬────────────┬─────────────┐
│ year ┆ cfs        ┆ gage_height │
│ ---  ┆ ---        ┆ ---         │
│ i32  ┆ f64        ┆ f64         │
╞══════╪════════════╪═════════════╡
│ 2018 ┆ 71.074642  ┆ 6.423744    │
│ 2004 ┆ 113.173465 ┆ NaN         │
│ 2016 ┆ 98.565988  ┆ 5.542945    │
│ 2002 ┆ 116.211979 ┆ NaN         │
│ 2013 ┆ 152.412803 ┆ 4.239074    │
│ …    ┆ …          ┆ …           │
│ 2006 ┆ 413.241603 ┆ NaN         │
│ 2011 ┆ 159.9936   ┆ 3.968769    │
│ 2015 ┆ 116.564455 ┆ 5.542621    │
│ 2017 ┆ 82.364265  ┆ 5.581915    │
│ 2005 ┆ 177.604843 ┆ NaN         │
└──────┴────────────┴─────────────┘


In [ ]:
%%timeit
>>> (dd
...  .with_columns(year=pl.col('datetime').dt.year())
...  .group_by('year', maintain_order=True)
...  .agg(pl.col(pl.Float64).mean())
...  .collect(engine='gpu')
...  )

94.1 ms ± 8.56 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
%%timeit
>>> (dd
...  .with_columns(year=pl.col('datetime').dt.year())
...  .group_by('year', maintain_order=True)
...  .agg(pl.col(pl.Float64).mean())
...  .collect()
...  )

93.4 ms ± 8.39 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
>>> print(dd
...  .set_sorted('datetime')
...  .group_by_dynamic(index_column='datetime', every='1y')
...  .agg(pl.col(pl.Float64).mean())
...  .collect(engine='gpu')
... )

shape: (20, 3)
┌──────────────────────────────┬────────────┬─────────────┐
│ datetime                     ┆ cfs        ┆ gage_height │
│ ---                          ┆ ---        ┆ ---         │
│ datetime[μs, America/Denver] ┆ f64        ┆ f64         │
╞══════════════════════════════╪════════════╪═════════════╡
│ 2001-01-01 00:00:00 MST      ┆ 74.304452  ┆ null        │
│ 2002-01-01 00:00:00 MST      ┆ 116.211979 ┆ null        │
│ 2003-01-01 00:00:00 MST      ┆ 86.957083  ┆ null        │
│ 2004-01-01 00:00:00 MST      ┆ 113.173465 ┆ null        │
│ 2005-01-01 00:00:00 MST      ┆ 177.604843 ┆ null        │
│ …                            ┆ …          ┆ …           │
│ 2016-01-01 00:00:00 MST      ┆ 98.565988  ┆ 5.542945    │
│ 2017-01-01 00:00:00 MST      ┆ 82.364265  ┆ 5.581915    │
│ 2018-01-01 00:00:00 MST      ┆ 71.074642  ┆ 6.423744    │
│ 2019-01-01 00:00:00 MST      ┆ 92.886454  ┆ 6.670158    │
│ 2020-01-01 00:00:00 MST      ┆ 58.426221  ┆ 6.470503    │
└────────────────────────

In [ ]:
## Window FUnctions
>>> print(dd
...  .set_sorted('datetime')
...  .group_by_dynamic(index_column='datetime', every='1mo')
...  .agg(pl.col('cfs').mean())
...  .with_columns(mean_cfs_3mo=pl.col('cfs').mean()
...                .rolling('datetime', period='3mo'))
...  .collect(engine='gpu')
... )

shape: (226, 3)
┌──────────────────────────────┬────────────┬──────────────┐
│ datetime                     ┆ cfs        ┆ mean_cfs_3mo │
│ ---                          ┆ ---        ┆ ---          │
│ datetime[μs, America/Denver] ┆ f64        ┆ f64          │
╞══════════════════════════════╪════════════╪══════════════╡
│ 2001-05-01 00:00:00 MDT      ┆ 48.8702    ┆ 48.8702      │
│ 2001-06-01 00:00:00 MDT      ┆ 22.946759  ┆ 35.90848     │
│ 2001-07-01 00:00:00 MDT      ┆ 18.0125    ┆ 29.943153    │
│ 2001-08-01 00:00:00 MDT      ┆ 248.004398 ┆ 96.321219    │
│ 2001-09-01 00:00:00 MDT      ┆ 29.152574  ┆ 98.389824    │
│ …                            ┆ …          ┆ …            │
│ 2020-05-01 00:00:00 MDT      ┆ 22.670635  ┆ 73.275465    │
│ 2020-06-01 00:00:00 MDT      ┆ 10.219805  ┆ 33.748688    │
│ 2020-07-01 00:00:00 MDT      ┆ 25.042315  ┆ 19.310918    │
│ 2020-08-01 00:00:00 MDT      ┆ 4.122728   ┆ 13.128283    │
│ 2020-09-01 00:00:00 MDT      ┆ 4.900842   ┆ 11.355295    │
└───────

In [ ]:
%%timeit
>>> (dd
...  .set_sorted('datetime')
...  .group_by_dynamic(index_column='datetime', every='1mo')
...  .agg(pl.col('cfs').mean())
...  .with_columns(mean_cfs_3mo=pl.col('cfs').mean()
...                .rolling('datetime', period='3mo'))
...  .collect(engine='gpu')
... )

89.1 ms ± 3.27 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
%%timeit
>>> (dd
...  .set_sorted('datetime')
...  .group_by_dynamic(index_column='datetime', every='1mo')
...  .agg(pl.col('cfs').mean())
...  .with_columns(mean_cfs_3mo=pl.col('cfs').mean()
...                .rolling('datetime', period='3mo'))
...  .collect()
... )

87.6 ms ± 6.16 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
!pip install -U "hvplot>=0.9.1"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 11.3 MB/s eta 0:00:00


In [ ]:
## Interpolation
>>> import datetime
>>> import pytz
>>> denver = pytz.timezone('America/Denver')
>>> jul_7 = datetime.datetime(2018, 7, 7).replace(tzinfo=denver)
>>> jul_9 = datetime.datetime(2018, 7, 9).replace(tzinfo=denver)

>>> offset = .1

>>> (dd
...  .filter(pl.col('datetime').is_between(jul_7, jul_9))
...  .with_columns(
...     'datetime', 'cfs',
...     fill0=pl.col('cfs').fill_null(0).add(offset),
...     interpolate=pl.col('cfs').interpolate().add(offset*2),
...     forward=pl.col('cfs').fill_null(strategy='forward').add(offset*3),
...     backward=pl.col('cfs').fill_null(strategy='backward').add(offset*4)
...  )
...  .collect(engine='gpu')
...   .plot(x='datetime', y=['cfs', 'fill0', 'interpolate', 'forward',
...                          'backward'])
...  ) # NO_OUTPUT

ModuleUpgradeRequiredError: hvplot>=0.9.1 is required for `.plot`

In [ ]:
## Joining
>>> url = 'https://github.com/mattharrison/datasets/raw/master/data/'\
...       'hanksville.csv'

>>> def tweak_temp(df_):
...     return (df_
...         .select(pl.col('DATE').str.to_datetime()
...                   .dt.replace_time_zone('America/Denver'),
...                 'PRCP', 'TMIN', 'TMAX', 'TOBS')
...     )

>>> raw_temp = pl.scan_csv(url)
>>> print(tweak_temp(raw_temp))

naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan)

 SELECT [col("DATE").str.strptime([String(raise)]).dt.replace_time_zone([String(raise)]), col("PRCP"), col("TMIN"), col("TMAX"), col("TOBS")] FROM
  Csv SCAN [https://github.com/mattharrison/datasets/raw/master/data/hanksville.csv]
  PROJECT */20 COLUMNS


In [ ]:
>>> dd_daily = (dd
...  .set_sorted('datetime')
...  .group_by_dynamic(index_column='datetime', every='1d')
...  .agg(pl.col('gage_height', 'cfs').mean())
...  .collect(engine='gpu')
... )

>>> print(dd_daily)

shape: (5_757, 3)
┌──────────────────────────────┬─────────────┬───────────┐
│ datetime                     ┆ gage_height ┆ cfs       │
│ ---                          ┆ ---         ┆ ---       │
│ datetime[μs, America/Denver] ┆ f64         ┆ f64       │
╞══════════════════════════════╪═════════════╪═══════════╡
│ 2001-05-07 00:00:00 MDT      ┆ null        ┆ 71.076087 │
│ 2001-05-08 00:00:00 MDT      ┆ null        ┆ 69.208333 │
│ 2001-05-09 00:00:00 MDT      ┆ null        ┆ 61.197917 │
│ 2001-05-10 00:00:00 MDT      ┆ null        ┆ 55.041667 │
│ 2001-05-11 00:00:00 MDT      ┆ null        ┆ 54.78125  │
│ …                            ┆ …           ┆ …         │
│ 2020-09-24 00:00:00 MDT      ┆ 6.156042    ┆ 9.410208  │
│ 2020-09-25 00:00:00 MDT      ┆ 6.175208    ┆ 10.041042 │
│ 2020-09-26 00:00:00 MDT      ┆ 6.204896    ┆ 11.062292 │
│ 2020-09-27 00:00:00 MDT      ┆ 6.179792    ┆ 10.200521 │
│ 2020-09-28 00:00:00 MDT      ┆ 6.160769    ┆ 9.555641  │
└──────────────────────────────┴──────

In [ ]:
>>> both = (tweak_temp(raw_temp).collect(engine='gpu')
...  .join(dd_daily, left_on='DATE', right_on='datetime',
...        validate='1:1')
...  )

>>> print(both)

shape: (5_056, 7)
┌──────────────────────────────┬──────┬──────┬──────┬──────┬─────────────┬───────────┐
│ DATE                         ┆ PRCP ┆ TMIN ┆ TMAX ┆ TOBS ┆ gage_height ┆ cfs       │
│ ---                          ┆ ---  ┆ ---  ┆ ---  ┆ ---  ┆ ---         ┆ ---       │
│ datetime[μs, America/Denver] ┆ f64  ┆ i64  ┆ i64  ┆ i64  ┆ f64         ┆ f64       │
╞══════════════════════════════╪══════╪══════╪══════╪══════╪═════════════╪═══════════╡
│ 2001-05-07 00:00:00 MDT      ┆ 0.0  ┆ 41   ┆ 82   ┆ 55   ┆ null        ┆ 71.076087 │
│ 2001-05-08 00:00:00 MDT      ┆ 0.0  ┆ 43   ┆ 85   ┆ 58   ┆ null        ┆ 69.208333 │
│ 2001-05-09 00:00:00 MDT      ┆ 0.0  ┆ 36   ┆ 92   ┆ 64   ┆ null        ┆ 61.197917 │
│ 2001-05-10 00:00:00 MDT      ┆ 0.0  ┆ 50   ┆ 92   ┆ 67   ┆ null        ┆ 55.041667 │
│ 2001-05-11 00:00:00 MDT      ┆ 0.0  ┆ 46   ┆ 87   ┆ 60   ┆ null        ┆ 54.78125  │
│ …                            ┆ …    ┆ …    ┆ …    ┆ …    ┆ …           ┆ …         │
│ 2020-09-20 00:00:00 MDT

# Exploring NVIDIA's dataset

https://colab.research.google.com/drive/1fWKej4oplAqHtHVdwTjgMywobo6yI7sT#scrollTo=986c74e0-94b7-48e4-9921-fb2a17a252f5

In [ ]:
if GOOGLE_COLAB:
    !wget https://storage.googleapis.com/rapidsai/polars-demo/transactions-t4-20.parquet -O transactions.parquet
else:
    !wget https://storage.googleapis.com/rapidsai/polars-demo/transactions.parquet -O transactions.parquet

!wget https://storage.googleapis.com/rapidsai/polars-demo/rainfall_data_2010_2020.csv

--2024-09-16 17:29:04--  https://storage.googleapis.com/rapidsai/polars-demo/transactions-t4-20.parquet
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.118.207, 74.125.200.207, 74.125.130.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.118.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 857658422 (818M) [application/octet-stream]
Saving to: ‘transactions.parquet’

transactions.parque 100%[===================>] 817.93M  14.7MB/s    in 51s     

2024-09-16 17:29:56 (15.9 MB/s) - ‘transactions.parquet’ saved [857658422/857658422]

--2024-09-16 17:29:56--  https://storage.googleapis.com/rapidsai/polars-demo/rainfall_data_2010_2020.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.12.207, 142.250.4.207, 172.253.118.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.12.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 131421 (128K) [

In [ ]:
transactions = pl.scan_parquet("transactions.parquet")
transactions.head(5).collect(engine='gpu')

CUST_ID,START_DATE,END_DATE,TRANS_ID,DATE,YEAR,MONTH,DAY,EXP_TYPE,AMOUNT
str,date,date,str,date,i64,i64,i64,str,f64
"""CI6XLYUMQK""",2015-05-01,null,"""T8I9ZB5A6X90UG8""",2015-09-11,2015,9,11,"""Motor/Travel""",20.27
"""CI6XLYUMQK""",2015-05-01,null,"""TZ4JSLS7SC7FO9H""",2017-02-08,2017,2,8,"""Motor/Travel""",12.85
"""CI6XLYUMQK""",2015-05-01,null,"""TTUKRDDJ6B6F42H""",2015-08-01,2015,8,1,"""Housing""",383.8
"""CI6XLYUMQK""",2015-05-01,null,"""TDUHFRUKGPPI6HD""",2019-03-16,2019,3,16,"""Entertainment""",5.72
"""CI6XLYUMQK""",2015-05-01,null,"""T0JBZHBMSVRFMMD""",2015-05-15,2015,5,15,"""Entertainment""",11.06


In [ ]:
%%timeit
transactions = pl.scan_parquet("transactions.parquet")
transactions.head(5).collect(engine='gpu')

46.7 ms ± 215 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
%%timeit
transactions = pl.scan_parquet("transactions.parquet")
transactions.head(5).collect()

11.7 ms ± 91.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
# min and max for each type
(transactions
 .group_by(['CUST_ID', 'EXP_TYPE'])
 .agg(min=pl.col('AMOUNT').min(), max=pl.col('AMOUNT').max())
 .sort(by=['EXP_TYPE', 'max'], descending=[True, True])
 .collect(engine='gpu')
)

CUST_ID,EXP_TYPE,min,max
str,str,f64,f64
"""CH8O40105K""","""Tax""",1332.06,1332.06
"""C93EOJSH4L""","""Tax""",1331.63,1331.63
"""CZ2R24C5TT""","""Tax""",1331.22,1331.22
"""C8A769WS2L""","""Tax""",1330.15,1330.15
"""C5FMCJLCR6""","""Tax""",1330.07,1330.07
…,…,…,…
"""CT975926GC""","""Bills and Utilities""",3.25,64.44
"""CYCOTPFEL2""","""Bills and Utilities""",2.76,64.17
"""CTI14FYGOR""","""Bills and Utilities""",2.83,63.97


In [ ]:
%%timeit
# min and max for each type
(transactions
 .group_by(['CUST_ID', 'EXP_TYPE'])
 .agg(min=pl.col('AMOUNT').min(), max=pl.col('AMOUNT').max())
 .sort(by=['EXP_TYPE', 'max'], descending=[True, True])
 .collect(engine='gpu')
)

145 ms ± 621 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
%%timeit
# min and max for each type
(transactions
 .group_by(['CUST_ID', 'EXP_TYPE'])
 .agg(min=pl.col('AMOUNT').min(), max=pl.col('AMOUNT').max())
 .sort(by=['EXP_TYPE', 'max'], descending=[True, True])
 .collect()
)

1.66 s ± 34.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
1_660 / 145

11.448275862068966

In [ ]:

!ls /content/drive/MyDrive/*.whl

/content/drive/MyDrive/cudf_polars_cu12-24.8.0a433-py3-none-any.whl


In [ ]:

# /content/drive/MyDrive/cudf_polars_cu12-24.8.0a433-py3-none-any.whl'
# /content/drive/MyDrive/cudf_polars_cu12-24.8.0a433-py3-none-any.whl

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
